In [162]:
import numpy as np
from typing import List, Dict, Tuple
import heapq
#global gas 
gas = 0
#def create_genome(n,lower_bound =0,upper_bound=10):
 #   return [np.random.randint(lower_bound,upper_bound) for _ in range(n)]
#create_genome(5)

def stochastic_cost_function_helper(x: list[int]) -> float:
    # Extract individual elements
    x1, x2, x3, x4, x5 = x
    
    # Check constraints
    if x1 + x2 + x3 > 20 or x4 + x5 != 20 or any(xi < 1 or xi > 20 for xi in x):
        return float('inf')  # Return a high cost for invalid configurations
    
    # Compute a hypothetical throughput based on service rates
    throughput = min(x1, x2, x3)  # Simplified throughput calculation
    
    # Penalize configurations that do not maximize throughput
    # Inverse of throughput is used as a cost
    cost = 1 / throughput if throughput > 0 else float('inf')
    
    # Add noise to simulate stochastic environment
    noise = np.random.normal(0, np.sqrt(cost))
    
    result_with_noise = cost + noise
    
    return result_with_noise

def stochastic_cost_function(x: list[int], n: int =10) -> float:
    global gas
    gas += n
    return np.mean([stochastic_cost_function_helper(x) for _ in range(n)])
    
def euclidean_distance(centre_1 : List[int],centre_2: List[int]):
    array1 = np.array(centre_1)
    array2 = np.array(centre_2)

    # Calculate Euclidean distance
    euclidean_distance = np.linalg.norm(array1 - array2)
    return euclidean_distance

def find_farthest_point(center: List[int], sol_space: List[List[int]]) -> Tuple[List[int], float]:
    max_distance = 0
    farthest_point = None
    
    for point in sol_space:
        # Calculate the squared distance
        squared_distance = sum((c - p)**2 for c, p in zip(center, point))
        
        if squared_distance > max_distance:
            max_distance = squared_distance
            farthest_point = point
    
    # Calculate the actual distance (square root of the squared distance)
    max_distance = np.sqrt(max_distance)
    
    return farthest_point, max_distance

def closeset_centre(centers:List[List[int]],visited_solution:List[int]) -> List[int]:
    closest = [float('inf') for _ in centers]
    min_dist= float('inf')
    for center in centers:
        dist =euclidean_distance(center,visited_solution)
        if dist < min_dist:
            min_dist = dist
            closest = center
        #print('hehe: ',closest)
    return closest

def initialize(n:int,mg:int,q:int) : #n is dimension of solution vector and mg is number of solns sampled
    global gas
    unique_set = {}
    while len(unique_set) <mg:
        genome = get_random_valid_vector()
        if tuple(genome) in unique_set:
            unique_set[tuple(genome)] = min(unique_set[tuple(genome)],stochastic_cost_function(genome))
        else:
            unique_set[tuple(genome)] = stochastic_cost_function(genome)
    heap = [(value,key) for key,value in unique_set.items()]
    heapq.heapify(heap)
    sorted_items = sorted(unique_set.items(), key=lambda x: x[1])
    niche_centers = sorted_items[:q]
    sol_space = sorted_items[q:]
    centers = [center[0] for center in niche_centers]
    clusters ={center:[] for center in centers}
    for sol in sol_space:
        clusters[tuple(closeset_centre(centers,sol[0]))].append(sol[0])
    best_center =  centers[0]
    sol_space = clusters[best_center]
    #print('Cluster: ',clusters)
    farthest_point , min_radius = find_farthest_point(best_center,centers[1:])
    
    for center in centers[1:]:
        if euclidean_distance(center,best_center) < 0.5*min_radius:
            clusters[tuple(best_center)].append(center)
            clusters[tuple(best_center)].append(clusters.pop(center)[0])
    centers_remaining = [list(key) for key in clusters.keys()][1:]
    r = float('inf')
    for center in centers_remaining:
        r = min(euclidean_distance(center,best_center),r)
    r = r//2
    q = len(clusters.keys())
    sol_vals_dict = {} # maps each solution to closest center

# Loop through the original dictionary
   # print('clus: ',clusters)
    for key, list_of_lists in clusters.items():
       # print('lol: ',list_of_lists)
        for lst in list_of_lists:
            # Convert the list to a tuple to make it hashable (since lists cannot be dictionary keys)
           # print('lst: ',lst)
            sol_vals_dict[tuple(lst)] = unique_set[tuple(lst)]

    center_vals ={}
    for center in clusters.keys():
        center_vals[center] = stochastic_cost_function(center)
    return clusters , q , r , sol_vals_dict , center_vals

def get_random_valid_vector(lower_bound: int = 1, upper_bound: int = 20) -> List[int]:
    while True:
        x1 = np.random.randint(lower_bound, upper_bound + 1)
        x2 = np.random.randint(lower_bound, upper_bound + 1)
        x3 = np.random.randint(lower_bound, upper_bound + 1)
        x4 = np.random.randint(lower_bound, upper_bound + 1)
        x5 = 20 - x4  # Ensure x4 + x5 = 20
        
        vector = [x1, x2, x3, x4, x5]
        
        if x1 + x2 + x3 <= 20 and 1 <= x5 <= 20 and stochastic_cost_function_helper(vector) != float('inf'):
            return vector

def get_n_random_valid_vectors(n: int, lower_bound: int = 1, upper_bound: int = 20) -> List[List[int]]:
    valid_vectors = []
    for _ in range(n):
        valid_vectors.append(get_random_valid_vector(lower_bound, upper_bound))
    return valid_vectors

    
clusters,q,r,sol_vals_dict,center_vals = initialize(5,400,4)


In [163]:


def select_random_pair_in_range(dictionary: Dict[Tuple[int], int], lower_bound: float, upper_bound: float):
    eligible_pairs = [(key, value) for key, value in dictionary.items() 
                      if lower_bound <= value <= upper_bound]
    if not eligible_pairs:
        return None  # No pairs found within the specified range
    
    return eligible_pairs[np.random.choice(len(eligible_pairs))][0]

def get_mate(genome: List[int], sol_val_dict: Dict[Tuple[int], int]):
    temp_dict = sol_val_dict.copy()  # Create a copy of the original dictionary
    cost_value = temp_dict[tuple(genome)]  # Get the cost value for the genome
    if cost_value >99999:
        print('genome: ',genome)
    del temp_dict[tuple(genome)]  # Remove the genome from the dictionary
    lower_bound = 1
    upper_bound = 20
    alpha = ((lower_bound + upper_bound)//2)*0.1
    mate = select_random_pair_in_range(temp_dict, (1 - alpha) * cost_value, (1 + alpha) * cost_value)
    print('mate:',cost_value)
    runs = 0
    while  mate == None or list(mate) == genome:
        #print('yeet', alpha)
        if runs >50:
            print('nononono')
            return None
        alpha *= 1.1
        mate = select_random_pair_in_range(temp_dict, (1 - alpha) * cost_value, (1 + alpha) * cost_value)
        runs +=1
    
    return list(mate)

def single_point_crossover(parent1, parent2):
    size = len(parent1)
    parent1 = tuple(parent1)
    parent2 = tuple(parent2)
    # Choose a random crossover point (excluding the first and last positions)
    crossover_point = np.random.randint(1, size - 2)
    
    # Create the first child
    child1 = list(parent1[:crossover_point] + tuple([-1] * (size - crossover_point)))
    pointer1 = crossover_point
    for gene in parent2[crossover_point:] + parent2[:crossover_point]:
        if gene not in child1:
            child1[pointer1] = gene
            pointer1 += 1
            if pointer1 == size:
                pointer1 = 0
    
    # Create the second child
    child2 = list(parent2[:crossover_point] + tuple([-1] * (size - crossover_point)))
    pointer2 = crossover_point
    for gene in parent1[crossover_point:] + parent1[:crossover_point]:
        if gene not in child2:
            child2[pointer2] = gene
            pointer2 += 1
            if pointer2 == size:
                pointer2 = 0
    
    return child1, child2

def evolution(clusters,sol_vals_dict,center_vals_dict):
    global gas
    centers = center_vals_dict.keys()
    centers = [list(center) for center in centers]
    center_vals = center_vals_dict.values()
    center_vals = [val for val in center_vals]
    while True:
        sorted_items = sorted(sol_vals_dict.items(), key=lambda x: x[1])
        #elite_genome = sorted_items[:5]
        other_genomes = sorted_items
        #elite_genomes = [temp[0] for temp in elite_genome]
        other_genomes = [list(genome[0]) for genome in other_genomes]    
            
        
        #print('pther_genomes: ',other_genomes)
        #print('centers',centers)
        m =len(other_genomes)
        sol_val_new ={}
        sol_space=[]
        unique_set= set()
        #print('lmao')
        print('m: ',m)
        for _ in range(m):
            i =np.random.randint(0,m)
           # print(_)
            parent1 = other_genomes[i]
            #print(parent1,'li')
            #print(sol_vals_dict,'sk')
            while stochastic_cost_function( parent1,10) == float('inf'):
                i =np.random.randint(0,m)
                parent1 = other_genomes[i]
            parent2 = get_mate(parent1,sol_vals_dict)
            #print('hehe')
            if parent2 == None:
                parent2 = parent1
           # print('asdas',parent2)
            child1,child2 = single_point_crossover(parent1,parent2)
            #print('lolo')
            cost_1 = stochastic_cost_function(child1)
            cost_2 = stochastic_cost_function(child2)
            add_1_to_sol = True
            add_2_to_sol = True
            unique_set.add(tuple(child1))
            unique_set.add(tuple(child2))
            #print('center_vals: ',center_vals)
            for j in range(len(center_vals)):
                #print('j',j)
                if center_vals[j] > cost_1:
                    add_1_to_sol = False
                    centers[j] = child1
                    center_vals[j] = cost_1
                if center_vals[j] > cost_2:
                    centers[j] = child2
                    center_vals[j] = cost_2
                    add_2_to_sol = False
            if add_1_to_sol:
                sol_val_new[tuple(child1)] = cost_1
                sol_space.append(child1)
            if add_2_to_sol:
                sol_val_new[tuple(child2)] = cost_2
                sol_space.append(child2)
        #print('099')
        centers1 = [tuple(center) for center in centers]
        clusters ={center:[] for center in centers1}
        for sol in sol_space:
            clusters[tuple(closeset_centre(centers1,sol[0]))].append(sol)
        best_center =  centers1[0]
        sol_space = clusters[best_center]
        #print('Cluster: ',clusters)
        #print(centers)
        farthest_point , min_radius = find_farthest_point(best_center,centers[1:])
        #print('sol_val:',sol_val_new)
        for center in centers[1:]:
            #print('eheh: ',center)
            if euclidean_distance(center,best_center) < 0.5*min_radius:
                clusters[tuple(best_center)].append(center)
                clusters[tuple(best_center)].append(clusters.pop(tuple(center))[0])
        #print('Clusters: ',clusters)
        sol_vals_dict = sol_val_new
        if len(centers) == 1:
            print("Only one center is present")
            print('no of centers: ',len(clusters))
            break
        print('gas:',gas)
        if gas > 50000:
            print('no of centers: ',len(clusters))
            print('budget over')
            break
    return clusters
        
            
            
            

In [164]:
np.random.seed(1234)


final_cluster_set=evolution(clusters,sol_vals_dict,center_vals)
print(final_cluster_set)

m:  212
mate: 0.2645219505554176
mate: 0.6178665238632493
mate: 0.2645219505554176
mate: 0.2628999331702138
mate: 0.2112500907608012
mate: 0.18081120000099973
mate: -0.01116364609844681
nononono
mate: 1.1803002571278636
mate: 1.097834953772897
mate: 0.14964168096879576
mate: 1.0678098559665568
mate: 1.3535046466166842
mate: 1.3535046466166842
mate: 0.19669074202371323
mate: 0.8222657076514477
mate: 0.2649296504366281
mate: 0.471532169236801
mate: 0.7751092470864951
mate: 0.13360630482329683
mate: 0.1072939823290012
mate: 0.15029811960715309
mate: 0.7226863185116916
mate: -0.006328490568736056
nononono
mate: 0.457173027406583
mate: 0.6178665238632493
mate: 0.39003578671575395
mate: 0.20630705195034144
mate: 0.4597948511961021
mate: 0.6559243856128618
mate: 0.20630705195034144
mate: 0.20630705195034144
mate: 0.9335898944039471
mate: 0.9092408182519559
mate: 0.7751092470864951
mate: 0.41031491677582005
mate: 1.5983620981822981
mate: 1.4427806716127853
mate: 0.1841333515214057
mate: 0.6347

In [165]:
centers_items =  final_cluster_set.keys()
centers = [list(i) for i in centers_items]
print(centers)
sol_items = final_cluster_set.values()
final_clusters =[]
#print(sol_items)
for i,local_cluster in enumerate(sol_items):
    print(local_cluster)
    local_cluster=local_cluster[:2]
    local_cluster.append(centers[i])
    final_clusters.append(local_cluster)
    print(final_clusters)
print(final_clusters)


[[5, 4, 9, 8, 12]]
[[2, 1, 6, 15, 5], [4, 6, 5, 7, 13], [2, 1, 6, 15, 5], [12, 1, 5, 15, 4], [1, 10, 8, 4, 16], [1, 9, 3, 15, 5], [6, 9, 2, 1, 19], [6, 1, 12, 14, 7], [11, 3, 6, 4, 16], [7, 10, 3, 2, 18], [2, 10, 5, 7, 13], [6, 7, 4, 8, 12], [3, 4, 5, 7, 13], [3, 2, 4, 1, 19], [8, 10, 2, 5, 15], [6, 4, 9, 15, 5], [2, 1, 4, 6, 14], [3, 1, 9, 5, 15], [3, 8, 9, 15, 5], [8, 3, 5, 18, 2], [1, 3, 7, 13, 6], [1, 3, 8, 7, 13], [4, 13, 2, 3, 17], [4, 13, 2, 3, 17], [4, 5, 7, 8, 12], [1, 2, 5, 8, 12], [1, 2, 16, 4, 8], [8, 2, 16, 4, 1], [11, 3, 5, 1, 19], [9, 2, 5, 8, 12], [2, 1, 5, 8, 12], [7, 1, 5, 16, 4], [5, 4, 7, 8, 12], [5, 11, 1, 19, 3], [8, 6, 1, 19, 5], [8, 6, 2, 7, 13], [14, 1, 4, 16, 3], [1, 14, 8, 17, 11], [4, 11, 5, 6, 14], [2, 11, 6, 14, 3], [2, 10, 5, 15, 8], [2, 11, 6, 15, 5], [1, 2, 4, 6, 14], [1, 5, 16, 4, 7], [1, 2, 16, 4, 8], [2, 1, 7, 13, 10], [4, 5, 7, 8, 12], [6, 5, 13, 7, 9], [7, 2, 9, 15, 5], [8, 10, 2, 5, 15], [3, 1, 13, 17, 4], [3, 1, 8, 5, 15], [2, 1, 5, 8, 12], [2, 5

COMPASS PART

import heapq
from typing import Tuple, List
import numpy as np
import math

def get_n_neighbours(x: List[int], n: int, lower_bound: int =1, upper_bound: int = 20) -> List[List[int]]:
    neighbors = []
    for _ in range(n):
        mean = 0
        std = 1
        random_floats = mean + std * np.random.randn(len(x))
        reach = np.round(random_floats).astype(int)
        
        # Create a new neighbor and ensure all values are within bounds
        new_neighbor = []
        for i, val in enumerate(x):
            new_val = val + reach[i]
            new_val = max(lower_bound, min(new_val, upper_bound))  # Clamp value between lower and upper bounds
            new_neighbor.append(new_val)
        
        neighbors.append(new_neighbor)
    
    return neighbors
def SAR(k):
    n0=8
    return math.ceil(n0*(np.log(k)**2))

get_n_neighbours([10,10],8)

from typing import List, Dict, Tuple
def euclidean_distance(list1: List[int], list2: List[int]) -> float:
    if len(list1) != len(list2):
        raise ValueError("Both lists must have the same length")
    
    sum_squared_diff = sum((a - b) ** 2 for a, b in zip(list1, list2))
    distance = math.sqrt(sum_squared_diff)
    return distance
from heapq import nsmallest

def find_closest_keys(x_star_k: List[int], V_k: Dict[Tuple[int, ...], float], n: int = 10) -> List[Tuple[int, ...]]:
    def euclidean_distance(a, b):
        return np.sqrt(sum((x - y) ** 2 for x, y in zip(a, b)))
    
    x_star_k_tuple = tuple(x_star_k)
    
    distances = [
        (key, euclidean_distance(x_star_k, key)) 
        for key in V_k.keys() 
        if key != x_star_k_tuple
    ]
    
    closest_keys = nsmallest(n, distances, key=lambda x: x[1])
    return [key for key, _ in closest_keys]

def get_most_promising_area(x_star_k:List[int],neighbours: List[List[int]], V_k: Dict[Tuple[int,...], float]):
    mp_area=[x_star_k]
    closest_keys = find_closest_keys(x_star_k,V_k,10)
    if len(V_k.keys()) ==0:
        return neighbours
    for sol in neighbours:
        sol_distance = euclidean_distance(x_star_k, sol)
        
        # Check if sol is closer to x_star_k than the other closest keys
        is_closer = all(sol_distance < euclidean_distance(x_star_k, key) for key in closest_keys)
        
        if is_closer:
            mp_area.append(sol)
    
    return mp_area
import gurobipy as gp
from gurobipy import GRB
def check_redundancy(x_star_k : List[int], V_k:List[List[int]]):
    x_star_k = np.array(x_star_k)
    redundancy_status = []
    
    for x_i in V_k:
        x_i = np.array(x_i)
        model = gp.Model()
        model.setParam('OutputFlag', 0)
        #print('x_star: ',x_star_k)
        n = len(x_star_k)
        x = model.addMVar(shape=n, vtype=GRB.CONTINUOUS, name="x")
        
        midpoint_i = (x_star_k + x_i) / 2
        diff = x_star_k - x_i
        
        # Correct way to create the objective function
        objective = gp.LinExpr()
        for j in range(n):
            #print('j:,',j,diff)
            objective += diff[j] * (x[j] - midpoint_i[j])
        model.setObjective(objective, GRB.MINIMIZE)
        
        for x_j in V_k:
            if not np.array_equal(np.array(x_j), x_i):
                x_j = np.array(x_j)
                midpoint_j = (x_star_k + x_j) / 2
                # Correct way to create the constraint
                constraint = gp.LinExpr()
                for j in range(n):
                    #print('j: ',j)
                    constraint += (x_star_k[j] - x_j[j]) * (x[j] - midpoint_j[j])
                model.addConstr(constraint >= 0)
        
        model.optimize()
        
        if model.status == GRB.OPTIMAL:
            obj_value = model.ObjVal
            redundancy_status.append(obj_value >= 0)
        else:
            redundancy_status.append(False)
    
    return redundancy_status
from multiprocessing import heap
from typing import List


def find_closest_keys(x_star_k: List[int], V_k: List[List[int]], n: int = 10) -> List[List[int]]:
    def euclidean_distance(a, b):
        return np.sqrt(sum((x - y) ** 2 for x, y in zip(a, b)))
    
    #x_star_k_tuple = tuple(x_star_k)
    
    distances = [
        (key, euclidean_distance(x_star_k, key)) 
        for key in V_k 
        if key != x_star_k
    ]
    
    closest_keys = nsmallest(n, distances, key=lambda x: x[1])
    return [key for key,_ in closest_keys]

def get_most_promising_area(x_star_k:List[int],neighbours: List[List[int]], V_k: List[List[int]]):
    mp_area=[]
    closest_keys = find_closest_keys(x_star_k,V_k,10)
    if len(V_k) ==0:
        return neighbours
    for sol in neighbours:
        sol_distance = euclidean_distance(x_star_k, sol)
        
        # Check if sol is closer to x_star_k than the other closest keys
        is_closer = all(sol_distance < euclidean_distance(x_star_k, key) for key in closest_keys)
        
        if is_closer:
            mp_area.append(sol)
    
    return mp_area

import random

def randomly_sample_solution(population: List[List[int]],ml:int):
    ml = min(ml,len(population))
    return random.sample(population,ml)

    
global global_store
global global_reps
global_store={}
global_reps ={}

def simulate(population: List[List[int]]): # here population refers to the niche being explored
    temp_heap=[]
    gas = 0
    k=2
    #global global_store
    
    for i in population:
        temp_heap.append((stochastic_cost_function(i,10),i))
        
    for i in temp_heap:
        solution = tuple(i[1])
        if solution in global_store:
            global_store[solution] = min(global_store[solution], i[0])
            global_reps[solution] = global_reps.get(solution, 0) + 1
        else:
            global_store[solution] = i[0]
            global_reps[solution] = 1
        
    heapq.heapify(temp_heap)
    x_star_k_temp= heapq.heappop(temp_heap)
    x_star_k = x_star_k_temp[1]
    x_star_k_val = x_star_k_temp[0]
    heap=[x_star_k_temp]
    heapq.heapify(heap)
    V_k = population
    population.remove(x_star_k)
    if len(population) >0:
        for sol in population:
            bound = ((10**20)**(1/len(sol)))/2
            #bound = ((10**20)**(1/4))/2 
            #if k ==2:
                #print('bound +-: ',bound)
                
            mp_area = get_most_promising_area(sol,get_n_neighbours(sol,10,lower_bound=-0,upper_bound=bound),V_k) #most promissing area
            while(len(mp_area)<1):
                mp_area = get_most_promising_area(sol,get_n_neighbours(sol,10,lower_bound=-0,upper_bound=bound),V_k)
            mp_area_vals=[ stochastic_cost_function(sol,SAR(k)) for i in mp_area]
            gas += SAR(k) * len(mp_area_vals)
            
     
        #print('first mp area: ',mp_area)
    else:
        bound = 20
        mp_area = get_most_promising_area(x_star_k,get_n_neighbours(x_star_k,10,lower_bound=0,upper_bound=bound),V_k)
        mp_area_vals=[ stochastic_cost_function(x_star_k,SAR(k)) for i in mp_area]
    for i in range(len(mp_area)):
        solution = tuple(mp_area[i])
        if solution in global_store:
            global_store[solution] = min(global_store[solution], mp_area_vals[i])
            global_reps[solution] = global_reps.get(solution, 0) + 1
        else:
            global_store[solution] = mp_area_vals[i]
            global_reps[solution] = 1
    running = True
    xk_store=[]
    while running:
        V_k += randomly_sample_solution(mp_area,100)
        temp =[]
        for i in range(len(V_k)):
            temp.append((stochastic_cost_function(V_k[i],SAR(k)),V_k[i]))
            gas += SAR(k)
       # print('V_k before prunning: ',V_k)
        for i in temp:
            solution = tuple(i[1])
            if solution in global_store:
                global_store[solution] = min(global_store[solution], i[0])
                global_reps[solution] = global_reps.get(solution, 0) + 1
            else:
                global_store[solution] = i[0]
                global_reps[solution] = 1
        heapq.heapify(temp)
        x_star_k_temp = heapq.heappop(temp)
        x_star_k_val = x_star_k_temp[0]
        x_star_k = x_star_k_temp[1]
        V_k_temp = V_k.copy()
        V_k_temp.remove(x_star_k)
        redundant_variable = check_redundancy(x_star_k,V_k_temp)
        V_k=[x_star_k]
        for i in range(len(redundant_variable)):
            if redundant_variable[i]:
                V_k.append(V_k_temp[i])
       # print('V_k post prunning: ',V_k)
        mp_area = get_most_promising_area(x_star_k,get_n_neighbours(x_star_k,10,lower_bound = 1,upper_bound=bound),V_k)
        mp_area_with_vals = [(mp_area[i],stochastic_cost_function(mp_area[i],10)) for i in range(len(mp_area))]
        #print('mp area: ',mp_area_with_vals)
        k+=1
        count = 0
        while len(mp_area) == 0 and count <=10:
            #print('hehe: ',mp_area)
            mp_area = get_most_promising_area(x_star_k,get_n_neighbours(x_star_k,10,lower_bound=1,upper_bound=bound),V_k)
            count +=1
        if len(mp_area) ==0 :
            running = False
        if gas >50000:
            print('Budget Over')
            running = False
        xk_store.append(x_star_k_val)
        #print('lol')
    return (x_star_k,x_star_k_val,xk_store)


NGA TO COMPASS API

In [167]:
Outputs =[]
import time
start  = time.time()
for cluster in final_clusters:
    
    Outputs.append(simulate(cluster[:5]))
print("Time Taken: ", time.time() - start)

Budget Over
Time Taken:  3.3143551349639893


In [168]:
Outputs

[([6, 6, 6, 12, 8],
  0.1816241751462918,
  [-0.11428620554342239,
   -0.23928200619665776,
   0.08323408734066816,
   0.1382733816076621,
   0.19677291156919757,
   0.1502023157090863,
   0.20863694356685197,
   0.10822628224433473,
   0.3444949928162266,
   0.24368231472965168,
   0.1417318982038057,
   0.15296572738384345,
   0.22850040610592187,
   0.18651923726587136,
   0.1377556781296933,
   0.1566674321402607,
   0.22732004474061918,
   0.20769113842609666,
   0.23691081055823193,
   0.26703273536144284,
   0.2667843330070706,
   0.1687704381241303,
   0.20222382836672767,
   0.19266677896087317,
   0.1404172631318029,
   0.17847086473104196,
   0.22230571202048982,
   0.21498220485057531,
   0.2908087587007687,
   0.21856546958424233,
   0.3251853169871465,
   0.3792395241849789,
   0.2549654474328983,
   0.23719394676628397,
   0.24556147471672818,
   0.21162956537066033,
   0.2204296614560136,
   0.21582236192743165,
   0.15380706947451966,
   0.23567597206486837,
   0.12085

In [169]:
final_clusters

[[[2, 1, 6, 15, 5], [4, 6, 5, 7, 13], [5, 4, 9, 8, 12]]]

In [170]:
for i in final_clusters[0]:
    print(i,stochastic_cost_function(i,10))

[2, 1, 6, 15, 5] 0.928222509855177
[4, 6, 5, 7, 13] 0.49913631424047067
[5, 4, 9, 8, 12] 0.3295455575724609


NameError: name 'f' is not defined